In [8]:
import nltk
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



In [9]:
import nltk.stem.porter

def tokenize(text):
    return nltk.word_tokenize(text)


def stem(word):
    return nltk.stem.porter.PorterStemmer().stem(word)

def bagOfWords(token,allWords):
    tokenizeWords = [stem(word) for word in token]
    bag = np.zeros(len(allWords),dtype=np.float32)
    for idx ,word in enumerate(allWords):
        if word in tokenizeWords:
            bag[idx] = 1.0
    return bag


In [10]:
allWords = []
tags = []
xY = []
with open("./intents.json") as f :
    data = json.load(f)

for intent in data['intents']:
    tag  = intent["tag"]
    tags.append(tag)
    for pattern in intent["patterns"]:
        word = tokenize(pattern)
        allWords.extend(word)
        xY.append((word, tag))



In [11]:
ignoreWords = ['?','!','.',',']
allWords = [stem(word) for word in allWords if word not in ignoreWords]
allWords = sorted(set(allWords))
tags = sorted(set(tags))
tags


['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']

In [12]:

xTrain ,yTrain = [] , []
for (x,y) in xY:
    bag = bagOfWords(x,allWords)
    xTrain.append(bag)
    yTrain.append(tags.index(y)) 
xTrain = np.array(xTrain)
yTrain = np.array(yTrain)
xTrain


array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.]], dtype=float32)

In [13]:
class Datasets(Dataset):
    def __init__(self):
        self.numberOfClasses = len(xTrain)
        self.xTrain = xTrain
        self.yTrain = yTrain
    def __getitem__(self, index):
        return (self.xTrain[index] , self.yTrain[index])
    def __len__(self):
        return self.numberOfClasses

dataSet = Datasets()
next(iter(dataSet))



(array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], dtype=float32),
 3)

In [14]:
trainLoader = DataLoader(dataSet,batch_size=32)


In [15]:
class Model(nn.Module):
    def __init__(self,input,hiddenSize,numClasses):
        super().__init__()
        self.Liner1 = nn.Linear(input,hiddenSize)
        self.Liner2 = nn.Linear(hiddenSize,hiddenSize)
        self.Liner3 = nn.Linear(hiddenSize,numClasses)
        self.Relu = nn.ReLU()
    def forward(self,x):
        x = self.Relu(self.Liner1(x))
        x = self.Relu(self.Liner2(x))
        x = self.Liner3(x)
        return x





In [16]:
len(xTrain[0])


55

In [17]:
xTrain.shape


(26, 55)

In [18]:

model = Model(55,256,7).to("cuda")
lossFun = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(),lr=0.001)


In [19]:
for epoch in range(500):
    for x, y in trainLoader:
        x, y = x.to("cuda"), y.to("cuda")
        optim.zero_grad()
        outputs = model(x)
        loss = lossFun(outputs, y)
        loss.backward()
        optim.step()

print(f' Loss: {loss.item():.4f}')




 Loss: 0.0001


In [20]:
torch.save(model,"model.pt")


In [21]:
Models = torch.load("./model.pt")
Models


Model(
  (Liner1): Linear(in_features=55, out_features=256, bias=True)
  (Liner2): Linear(in_features=256, out_features=256, bias=True)
  (Liner3): Linear(in_features=256, out_features=7, bias=True)
  (Relu): ReLU()
)

In [22]:
import random


In [23]:
Models.eval()
botName = "HaSoNa"
print("let's go type quit for exit")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    message = tokenize(message)
    x = bagOfWords(message,allWords)
    x = x.reshape(1,x.shape[0])
    x = torch.tensor(x).to("cuda")
    output = Models(x)
    _ , predicted = torch.max(output,1)
    tag = tags[predicted.item()]
    probs = torch.softmax(output,dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in data["intents"]:
            if tag == intent["tag"]:
                print(f"{botName} : {random.choice(intent['responses'])}")
    else :\
        print(f"{botName} : I didn't understand that")


let's go type quit for exit
HaSoNa : Hey :-)
HaSoNa : What did the buffalo say when his son left for college? Bison.
HaSoNa : I didn't understand that
